In [1]:
import pandas as pd
df = pd.read_csv('train.csv', quoting=1, on_bad_lines='skip', engine='python')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(1711, 5)

In [3]:
df.isnull().sum()

id          0
title      44
author    185
text        3
label       0
dtype: int64

We can see that there are many null values in the dataset. We cannot relace the values in it by any values as it is text dataset. So, we can drop the null values here.

In [4]:
# Dropping NaN values...
df = df.dropna()

In [5]:
df.shape
# The shape got reduced...

(1482, 5)

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# Separating the dependent & indepndent features...
X = df.drop('label', axis=1)

In [8]:
y = df['label']

In [9]:
print(X.shape)

(1482, 4)


In [10]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
# Vocabulary size...
voc_size = 5000

### One hot representation...

In [13]:
# Making a copy of the independent features to work on...
messages = X.copy()

In [14]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [15]:
# Now, applying stemming & stopwords...
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
messages.reset_index(inplace=True)

In [18]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
1477,1705,1705,5.0 Quake Near Oklahoma Oil Hub Causes Substan...,Alex Ansary,5.0 Quake Near Oklahoma Oil Hub Causes Substan...
1478,1706,1706,"Islamic State Again Strikes at Afghan Shiites,...",Mujib Mashal and Fahim Abed,"KABUL, Afghanistan — A suicide bomber struc..."
1479,1707,1707,WaPo Report: Mueller Investigating Donald Trum...,Michelle Moons,Recently appointed Special Counsel on the Russ...
1480,1708,1708,Alabama Makes It Illegal to Remove Confederate...,Ryan Saavedra,The new law attempts to preserve history by ma...


In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    # Applying Stopwords...
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4041, 4634, 4149, 1708, 4548, 4741, 3505, 378, 110, 796],
 [4590, 1825, 1085, 1370, 1247, 4900, 3342],
 [2137, 4055, 864, 1022],
 [2657, 552, 4478, 3390, 2342, 4449],
 [4181, 1247, 3515, 4265, 2958, 2280, 1247, 701, 1600, 4847],
 [2734,
  3211,
  1230,
  1517,
  407,
  1434,
  1180,
  2138,
  1454,
  4291,
  2810,
  4252,
  3825,
  1573,
  3342],
 [3, 4611, 3991, 1973, 1643, 4494, 2004, 4762, 3017, 1478, 1791],
 [1422, 494, 286, 3712, 4320, 1845, 1434, 3576, 3017, 1478, 1791],
 [100, 563, 2261, 2663, 2397, 2649, 862, 2352, 1434, 3030],
 [4034, 4166, 2107, 4536, 4917, 1932, 4386, 4246],
 [4938, 357, 1120, 3959, 1530, 172, 4928, 1522, 4142, 3934, 3424],
 [3390, 806, 4548, 2649, 1434, 4320],
 [4591, 3547, 1135, 2014, 1759, 336, 2893, 2065, 4447],
 [3650, 2997, 4799, 2134, 2624, 3212, 4831, 3017, 1478, 1791],
 [1018, 4109, 2563, 4391, 1869, 3017, 1478, 1791],
 [3403, 2599, 4888, 1461, 1684, 1837, 2666, 3289, 1129, 4492],
 [4416, 425, 1825],
 [1594, 4411, 546, 852, 1434, 3716, 4802, 3342]

### Embedding Representation...

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4041 4634 4149 ...    0    0    0]
 [4590 1825 1085 ...    0    0    0]
 [2137 4055  864 ...    0    0    0]
 ...
 [2077 2614 4562 ...    0    0    0]
 [ 748 1821 1219 ...    0    0    0]
 [1599 2004 4060 ...    0    0    0]]


In [24]:
embedded_docs[1]

array([4590, 1825, 1085, 1370, 1247, 4900, 3342,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [25]:
embedded_docs[0]

array([4041, 4634, 4149, 1708, 4548, 4741, 3505,  378,  110,  796,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape

(1482, (1482,))

In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((1482, 20), (1482,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
16/16 [==============================] - 4s 96ms/step - loss: 0.6847 - accuracy: 0.5625 - val_loss: 0.6859 - val_accuracy: 0.5367
Epoch 2/10
16/16 [==============================] - 1s 67ms/step - loss: 0.5789 - accuracy: 0.6724 - val_loss: 0.5173 - val_accuracy: 0.7714
Epoch 3/10
16/16 [==============================] - 1s 57ms/step - loss: 0.2363 - accuracy: 0.9194 - val_loss: 0.5078 - val_accuracy: 0.7653
Epoch 4/10
16/16 [==============================] - 1s 36ms/step - loss: 0.1363 - accuracy: 0.9577 - val_loss: 0.5552 - val_accuracy: 0.8041
Epoch 5/10
16/16 [==============================] - 1s 35ms/step - loss: 0.0549 - accuracy: 0.9879 - val_loss: 0.5419 - val_accuracy: 0.8122
Epoch 6/10
16/16 [==============================] - 1s 36ms/step - loss: 0.0195 - accuracy: 0.9960 - val_loss: 0.6791 - val_accuracy: 0.8000
Epoch 7/10
16/16 [==============================] - 1s 36ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.7983 - val_accuracy: 0.8000
Epoch 8/10
16

In [32]:
y_pred=model.predict(X_test)

16/16 [==============================] - 1s 8ms/step


In [33]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[226,  37],
       [ 65, 162]])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7918367346938775

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       263
           1       0.81      0.71      0.76       227

    accuracy                           0.79       490
   macro avg       0.80      0.79      0.79       490
weighted avg       0.79      0.79      0.79       490

